In [1]:
import os
os.getcwd()
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from vae_earlystopping import EarlyStopping
from model.m2_bce import BCEcVAE
from model.m2_mse import MSEcVAE
from loss.l2_bce import l2_bce
from loss.l2_mse import l2_mse
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import json

In [ ]:


# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- train_and_evaluate 함수는 이전과 동일 (gamma_val 인자 사용) ---
def train_and_evaluate(c_path, x_data, seed, gamma_val):
    c_data = np.load(c_path)
    x_train, x_temp, c_train, c_temp = train_test_split(x_data, c_data, random_state=seed, test_size=0.4)
    x_val, x_test, c_val, c_test = train_test_split(x_temp, c_temp, random_state=seed, test_size=0.5)
    
    x_scaler, c_scaler = MinMaxScaler(), MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val, x_test = x_scaler.transform(x_val), x_scaler.transform(x_test)
    c_val, c_test = c_scaler.transform(c_val), c_scaler.transform(c_test)
    
    t_loader = DataLoader(TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(c_train)), batch_size=64, shuffle=True)
    v_loader = DataLoader(TensorDataset(torch.FloatTensor(x_val), torch.FloatTensor(c_val)), batch_size=64)
    test_loader = DataLoader(TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(c_test)), batch_size=64)
    
    x_dim, c_dim = x_train.shape[1], c_train.shape[1]

    # --- 1. BCE 모델 학습 ---
    model_bce = BCEcVAE(x_dim, c_dim, z_dim=8).to(device)
    opt_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
    es_bce = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, 801):
        model_bce.train()
        for x, c in t_loader:
            x, c = x.to(device), c.to(device)
            opt_bce.zero_grad()
            logit, mu, logvar = model_bce(x, c)
            loss = l2_bce(logit, x, mu, logvar, beta=1, gamma=gamma_val)['loss']
            loss.backward(); opt_bce.step()
        
        model_bce.eval()
        val_loss = 0
        with torch.no_grad():
            for vx, vc in v_loader:
                vx, vc = vx.to(device), vc.to(device)
                vl, vm, vv = model_bce(vx, vc)
                val_loss += l2_bce(vl, vx, vm, vv, beta=1, gamma=gamma_val)['loss'].item()
        if es_bce(val_loss/len(v_loader), model_bce): break
    es_bce.load_best_model(model_bce)

    # --- 2. MSE 모델 학습 ---
    model_mse = MSEcVAE(x_dim, c_dim, z_dim=8).to(device)
    opt_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
    es_mse = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, 801):
        model_mse.train()
        for x, c in t_loader:
            x, c = x.to(device), c.to(device)
            opt_mse.zero_grad()
            xh, mu, logvar = model_mse(x, c)
            loss = l2_mse(xh, x, mu, logvar, alpha=1, gamma=gamma_val)['loss']
            loss.backward(); opt_mse.step()
            
        model_mse.eval()
        val_loss = 0
        with torch.no_grad():
            for vx, vc in v_loader:
                vx, vc = vx.to(device), vc.to(device)
                xh, vm, vv = model_mse(vx, vc)
                val_loss += l2_mse(xh, vx, vm, vv, alpha=1, gamma=gamma_val)['loss'].item()
        if es_mse(val_loss/len(v_loader), model_mse): break
    es_mse.load_best_model(model_mse)

    # --- 3. 최종 평가 ---
    model_bce.eval(); model_mse.eval()
    all_bce_logits, all_mse_preds, all_true_scaled = [], [], []
    with torch.no_grad():
        for xt, ct in test_loader:
            xt, ct = xt.to(device), ct.to(device)
            b_logit, _, _ = model_bce(xt, ct)
            m_pred, _, _ = model_mse(xt, ct)
            all_bce_logits.append(b_logit.cpu().numpy())
            all_mse_preds.append(m_pred.cpu().numpy())
            all_true_scaled.append(xt.cpu().numpy())

    bce_logits, mse_preds_scaled, x_true_scaled = np.vstack(all_bce_logits), np.vstack(all_mse_preds), np.vstack(all_true_scaled)
    bce_prob = 1 / (1 + np.exp(-bce_logits)) 
    combined_scaled = mse_preds_scaled * bce_prob
    final_pred_phys = x_scaler.inverse_transform(combined_scaled)
    x_true_phys = x_scaler.inverse_transform(x_true_scaled)
    
    return float(r2_score(x_true_phys.flatten(), final_pred_phys.flatten()))

# --- 메인 통합 실험 루프 ---
x_data = np.load('./data/metal.npy')
gamma_list = [0.01,0.1,1,10]
seeds = np.random.randint(1, 100, size=20)

# 모든 결과를 담을 통합 딕셔너리
all_gamma_results = {}

for g in gamma_list:
    g_key = f"gamma_{str(g).replace('.', '_')}"
    all_gamma_results[g_key] = []
    print(f"\n>>> Running Experiment for Gamma: {g}")
    
    for i, seed in enumerate(seeds):
        r2_val = train_and_evaluate('./data/pre_re_fin_expand.npy', x_data, seed, g)
        all_gamma_results[g_key].append(r2_val)
        print(f"   [Seed {seed}] R2: {r2_val:.4f}")

# 최종 통합 저장
save_path = "./Combined_Gamma_Results.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump({
        "metadata": {"seeds": seeds.tolist(), "gamma_tested": gamma_list},
        "results": all_gamma_results
    }, f, indent=2, ensure_ascii=False)

print(f"\n✅ 모든 실험 완료! 결과가 {save_path}에 통합 저장되었습니다.")


>>> Running Experiment for Gamma: 0.01
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 7 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping c

C:\Users\mingyu\AppData\Local\Temp\ipykernel_10328\2820959577.py:83: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-bce_logits))


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counte

C:\Users\mingyu\AppData\Local\Temp\ipykernel_10328\2820959577.py:83: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-bce_logits))


   [Seed 92] R2: 0.9420
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of

...

   [Seed 13] R2: 0.8751
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of

C:\Users\mingyu\AppData\Local\Temp\ipykernel_10328\2820959577.py:83: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-bce_logits))


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 7 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counte

C:\Users\mingyu\AppData\Local\Temp\ipykernel_10328\2820959577.py:83: RuntimeWarning: overflow encountered in exp
  bce_prob = 1 / (1 + np.exp(-bce_logits))


EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counte